# Documentação: Resolução de Problema no Azure Storage + Databricks

## Contexto
Durante a execução de scripts automatizados no Databricks, um erro foi identificado ao tentar gravar dados no catálogo. Apesar do caminho existir no Azure Storage, o script falhava ao encontrar a pasta devido a um espaçamento inicial no nome do diretório, como " abcd".

Esse problema gerava inconsistências no pipeline, impossibilitando a gravação e o acesso correto aos dados.

## Solução Implementada
A solução seguiu os passos abaixo para garantir a resolução e evitar problemas futuros:

### 1. Listar Pastas no Diretório do Azure Storage
O primeiro passo foi listar todas as pastas no diretório do Azure Storage para verificar o problema e identificar os caminhos exatos com o comando dbutils.fs.ls():

In [ ]:
dbutils.fs.ls("Link/caminho abfss do storage")

### 2. Excluir Tabelas do Catálogo no Databricks
Antes de apagar as pastas no storage, foi necessário excluir as tabelas associadas no catálogo do Databricks. As tabelas foram especificadas em uma lista e removidas utilizando o comando DROP TABLE IF EXISTS:

In [ ]:
tabelas_para_excluir = [
    "catalogo.squema.tabelaA1",
    "catalogo.squema.tabelaB2",
    "catalogo.squema.tabelaC3"
]

# Exclui tabelas
for tabela in tabelas_para_excluir:
    spark.sql(f"DROP TABLE IF EXISTS {tabela}")


### 3. Verificar a Exclusão das Tabelas
Para garantir que as tabelas foram efetivamente excluídas, foi realizado um loop que verificava a existência de cada tabela no catálogo e registrava os resultados em um DataFrame:

In [ ]:
tabelas_excluidas = []
for tabela in tabelas_para_excluir:
    db, tbl = tabela.rsplit('.', 1)
    tables = spark.sql(f"SHOW TABLES IN {db}")
    table_exits = tables.filter(tables.tableName == tbl).count() > 0
    tabelas_excluidas.append({"Table": tabela, "Dropped": not table_exits})

# Cria um dataframe com o resultado
df_result = spark.createDataFrame(tabelas_excluidas)
display(df_result)


### 4. Excluir Pastas do Azure Storage
Após a exclusão das tabelas, as pastas problemáticas no Azure Storage foram removidas usando o comando dbutils.fs.rm():

In [ ]:
caminhos_para_excluir = [
    "caminho_para_ser_excluidoA1",
    "caminho_para_ser_excluidoB2",
    "caminho_para_ser_excluidoC3"
]

# Exclui pastas
for caminho in caminhos_para_excluir:
    dbutils.fs.rm(caminho, recurse=True)


### Observações e Melhorias Futuras
1-Validação de Nomes de Pastas: Implementar um script para validar nomes de diretórios automaticamente, removendo espaços ou caracteres inválidos antes da execução do pipeline. \n

2-Logs Detalhados: Adicionar logs em cada etapa do processo para rastreabilidade e depuração, registrando ações como exclusão de tabelas e remoção de pastas. \n

3-Automação do Pipeline: Criar uma função que combine todas as etapas descritas em um pipeline robusto e modular. \n

## Conclusão
Este processo garantiu a resolução do problema e evitou que o espaçamento nos nomes de pastas causasse erros no pipeline. A abordagem foi replicável e documentada para uso em situações semelhantes, contribuindo para maior eficiência e confiabilidade no ambiente de dados.